# NLU & NN - Implementação de Redes Neurais para Entendimento Natural de Texto

Atividade feita em sala de aula (23/08/24) com os seguintes integrantes do grupo 4 presencialmente:

- Allan Casado
- Cristiane Andrade
- Elias Biondo
- Giovana Thomé
- Melyssa Rojas
- Rafael Cabral

## Importação de bibliotecas e dos dados pré-processados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.17.0


In [ ]:
dataset_file_path = "/content/drive/MyDrive/Módulos/Módulo 11/data/Copy of processed_data.xlsx"
df = pd.read_excel(dataset_file_path)
df['processed_question'] = df['processed_question'].fillna('')

In [ ]:
df

,intention,question,processed_question
0,Acesso a conta,Boa tarde\nPor gentileza alguém pode me ajudar...,"gentileza,alguém,ajudar,preciso,enviar,hj,dinh..."
1,Acesso a conta,Atualizei os site e agora como acessar minha c...,"atualizei,site,acessar,conta"
2,Acesso a conta,Boa tarde estou tendo problema com o pin\neu n...,"ter,problema,pin,lembro,senhar,pin,consigo,tro..."
3,Acesso a conta,E a senha\nNão me lembro,"senhar,lembro"
4,Acesso a conta,Eu não consigo entrar na conta,"consigo,entrar,conta"
...,...,...,...
500,Termos e condicoes do servico,Bom dia por favor gostaria de saber qual o val...,"dia,gostar,remesso,Brasil"
501,Termos e condicoes do servico,Obg por informação.\npor favor informe-me o to...,"obg,informação,informemer,total,enviar,Brasil"
502,Termos e condicoes do servico,Oi queria Saber ate qto pode mandar na remessa...,"oi,querer,ate,qto,mandar,remesso,dinheiro"
503,Termos e condicoes do servico,"Olá, tudo bem?\nVocês enviam dinheiro do Japão...","olá,enviar,dinheiro,Japão,coreiar,sul"


## Encoding dos labels

In [ ]:
# Extraindo os valores únicos da coluna 'intention' para criar os labels

unique_intentions = df['intention'].unique().tolist()
print(f"length of unique_intentions: {len(unique_intentions)}")
print(f"unique_intentions: {unique_intentions}")

length of unique_intentions: 18
unique_intentions: ['Acesso a conta', 'Atualizacao de dados cadastrais', 'Cadastro de beneficiario', 'Cancelamento', 'Como depositar', 'Como fazer remessa', 'Como se inscrever', 'Confirmacao de cambio/taxas', 'Envio via Deposit Code', 'Pedido de envio via metodo "ByPhone"', 'Problemas de remessa', 'Reembolso', 'Registro/Atualizacao de Documento', 'Regras do servico', 'Solicitacao de cartao de remessas', 'Tempo de entrega do cartao', 'Tempo de remessa', 'Termos e condicoes do servico']


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Criação de uma coluna que mostra cada intenção como um inteiro (labels)
df['intention_label'] = label_encoder.fit_transform(df['intention'])

# Mapeamento dos labels das intenções
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'Acesso a conta': 0, 'Atualizacao de dados cadastrais': 1, 'Cadastro de beneficiario': 2, 'Cancelamento': 3, 'Como depositar': 4, 'Como fazer remessa': 5, 'Como se inscrever': 6, 'Confirmacao de cambio/taxas': 7, 'Envio via Deposit Code': 8, 'Pedido de envio via metodo "ByPhone"': 9, 'Problemas de remessa': 10, 'Reembolso': 11, 'Registro/Atualizacao de Documento': 12, 'Regras do servico': 13, 'Solicitacao de cartao de remessas': 14, 'Tempo de entrega do cartao': 15, 'Tempo de remessa': 16, 'Termos e condicoes do servico': 17}


In [ ]:
df

,intention,question,processed_question,intention_label
0,Acesso a conta,Boa tarde\nPor gentileza alguém pode me ajudar...,"gentileza,alguém,ajudar,preciso,enviar,hj,dinh...",0
1,Acesso a conta,Atualizei os site e agora como acessar minha c...,"atualizei,site,acessar,conta",0
2,Acesso a conta,Boa tarde estou tendo problema com o pin\neu n...,"ter,problema,pin,lembro,senhar,pin,consigo,tro...",0
3,Acesso a conta,E a senha\nNão me lembro,"senhar,lembro",0
4,Acesso a conta,Eu não consigo entrar na conta,"consigo,entrar,conta",0
...,...,...,...,...
500,Termos e condicoes do servico,Bom dia por favor gostaria de saber qual o val...,"dia,gostar,remesso,Brasil",17
501,Termos e condicoes do servico,Obg por informação.\npor favor informe-me o to...,"obg,informação,informemer,total,enviar,Brasil",17
502,Termos e condicoes do servico,Oi queria Saber ate qto pode mandar na remessa...,"oi,querer,ate,qto,mandar,remesso,dinheiro",17
503,Termos e condicoes do servico,"Olá, tudo bem?\nVocês enviam dinheiro do Japão...","olá,enviar,dinheiro,Japão,coreiar,sul",17


## Encoding e padding da coluna `question`

In [ ]:
# Definindo o tamanho do vocabulário com tokenização e Counter()
from collections import Counter

all_words = df['question'].str.split().sum()
vocabulary_size = len(Counter(all_words))

print(f"Vocabulary Size: {vocabulary_size}")

Vocabulary Size: 1572


In [ ]:
# Pegando o valor de extensão da maior frase do dataset


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 1600 # arredondado
input_length = 100 # jogando para cima caso alguma frase de validação seja maior

# Aplicando encoding e padding para a coluna 'question'
df['encoded_question'] = df['question'].apply(lambda x: one_hot(x, vocab_size))
df['encoded_question'] = pad_sequences(df['encoded_question'], maxlen=input_length, padding='post').tolist()

In [ ]:
df

,intention,question,processed_question,encoded_question,intention_label
0,Acesso a conta,Boa tarde\nPor gentileza alguém pode me ajudar...,"gentileza,alguém,ajudar,preciso,enviar,hj,dinh...","[1031, 757, 1595, 13, 810, 1377, 239, 1327, 15...",0
1,Acesso a conta,Atualizei os site e agora como acessar minha c...,"atualizei,site,acessar,conta","[878, 1072, 1244, 1326, 219, 1424, 277, 378, 1...",0
2,Acesso a conta,Boa tarde estou tendo problema com o pin\neu n...,"ter,problema,pin,lembro,senhar,pin,consigo,tro...","[1031, 757, 388, 315, 844, 579, 545, 458, 1264...",0
3,Acesso a conta,E a senha\nNão me lembro,"senhar,lembro","[1326, 375, 1303, 1166, 239, 888, 0, 0, 0, 0, ...",0
4,Acesso a conta,Eu não consigo entrar na conta,"consigo,entrar,conta","[1264, 1166, 1468, 92, 1343, 1074, 0, 0, 0, 0,...",0
...,...,...,...,...,...
500,Termos e condicoes do servico,Bom dia por favor gostaria de saber qual o val...,"dia,gostar,remesso,Brasil","[873, 1429, 1595, 304, 1296, 948, 979, 501, 54...",17
501,Termos e condicoes do servico,Obg por informação.\npor favor informe-me o to...,"obg,informação,informemer,total,enviar,Brasil","[349, 1595, 334, 1595, 304, 1588, 239, 545, 98...",17
502,Termos e condicoes do servico,Oi queria Saber ate qto pode mandar na remessa...,"oi,querer,ate,qto,mandar,remesso,dinheiro","[1197, 187, 979, 1114, 117, 1377, 880, 1343, 1...",17
503,Termos e condicoes do servico,"Olá, tudo bem?\nVocês enviam dinheiro do Japão...","olá,enviar,dinheiro,Japão,coreiar,sul","[323, 1255, 180, 441, 373, 1083, 621, 1249, 79...",17


## Definição do modelo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten

# Define the parameters
vocab_size = 1600  # The size of the vocabulary
input_length = 100  # The input length (max length of the sequences)
embedding_dim = 50  # The output dimension of the embedding vectors
num_classes = 18  # Number of classes

# Define the model
model = Sequential()

# Add the Embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length))

# Flatten the output from the Embedding layer
model.add(Flatten())

# Add a Dense layer with 128 units and ReLU activation (you can adjust the number of units)
model.add(Dense(128, activation='relu'))

# Add the output layer with 'num_classes' units and softmax activation for multi-class classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
df.shape

(505, 5)

## Treinamento

Considerando que a base de dados possui 505 frases pré-processadas, foi escolhido inicialmente a separação de aproximadamente 20% dos dados para teste e ~80% para treino. A separação não foi feita de maneira aleatória, foi feita separando as primeiras 400 frases para treinamento e o restante (105) para teste.

Essa separação de dados foi errônea, visto que os dados estavam separados e ordenados por categoria, o que resultou em uma acurácia inicial baixíssima.

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Assuming you already have the labels in a column 'intention_label'
# Convert the labels to categorical format
labels = to_categorical(df['intention_label'], num_classes=18)

# Split the data into training and testing sets
X_train = np.array(df['encoded_question'][:400].tolist())  # First 400 for training
y_train = labels[:400]

X_test = np.array(df['encoded_question'][400:].tolist())  # The rest for testing and validation
y_test = labels[400:]

# Compile the model with categorical_crossentropy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8501 - loss: 0.7036 - val_accuracy: 0.0000e+00 - val_loss: 8.7519
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8932 - loss: 0.5271 - val_accuracy: 0.0190 - val_loss: 8.9853
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9165 - loss: 0.4546 - val_accuracy: 0.0095 - val_loss: 10.8611
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9287 - loss: 0.3127 - val_accuracy: 0.0286 - val_loss: 10.7807
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9414 - loss: 0.3051 - val_accuracy: 0.0190 - val_loss: 11.8956
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9651 - loss: 0.2216 - val_accuracy: 0.0286 - val_loss: 11.6194
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9675 - loss: 0.1939 - val_accuracy: 0.0190 - val_loss: 12.1853
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9833 - loss: 0.1387 - val_acc

Com base na observação feita no treino anterior, os dados foram separados aleatoriamente em um segundo treino, aumentando consideravelmente a acurácia.

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Assuming you already have the labels in a column 'intention_label'
# Convert the labels to categorical format
labels = to_categorical(df['intention_label'], num_classes=18)

# Convert encoded questions to a NumPy array
X = np.array(df['encoded_question'].tolist())

# Split the data into training and testing sets with shuffling
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

# Compile the model with categorical_crossentropy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.7590 - loss: 1.4196 - val_accuracy: 0.7434 - val_loss: 0.9598
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8482 - loss: 0.5060 - val_accuracy: 0.7961 - val_loss: 0.8065
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8950 - loss: 0.3664 - val_accuracy: 0.7961 - val_loss: 0.6919
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9314 - loss: 0.2846 - val_accuracy: 0.8092 - val_loss: 0.6707
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9702 - loss: 0.1793 - val_accuracy: 0.7961 - val_loss: 0.7069
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9721 - loss: 0.1418 - val_accuracy: 0.8026 - val_loss: 0.6876
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9803 - loss: 0.1427 - val_accuracy: 0.7961 - val_loss: 0.6928
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9796 - loss: 0.1101 - val_accuracy: 0.

É importante ressaltar que a rede neural implementada não conhece outras palavras que não estejam no vocabulário de treino, ou seja, o resultado pode ser prejudicado se inseridas frases com palavras nunca vistas (isso SE a palavra receber um label numérico e entrar como input na rede neural)

## Teste individual

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Define your input sentence
new_sentence = "Eu não consigo entrar na conta"

# Define the vocabulary size and input length
vocab_size = 1600  # Adjust based on your model's vocab size
input_length = 100  # Adjust based on your model's input length

# Encode the sentence using one-hot encoding (assuming the same vocab_size)
encoded_sentence = one_hot(new_sentence, vocab_size)

# Pad the encoded sentence to match the input_length used in the model
padded_sentence = pad_sequences([encoded_sentence], maxlen=input_length, padding='post')

# Predict the class for the new sentence
prediction = model.predict(padded_sentence)

# Get the index of the highest probability
predicted_class = np.argmax(prediction, axis=1)

# Mapping of class indices to labels
class_labels = {
    0: 'Acesso a conta',
    1: 'Atualizacao de dados cadastrais',
    2: 'Cadastro de beneficiario',
    3: 'Cancelamento',
    4: 'Como depositar',
    5: 'Como fazer remessa',
    6: 'Como se inscrever',
    7: 'Confirmacao de cambio/taxas',
    8: 'Envio via Deposit Code',
    9: 'Pedido de envio via metodo "ByPhone"',
    10: 'Problemas de remessa',
    11: 'Reembolso',
    12: 'Registro/Atualizacao de Documento',
    13: 'Regras do servico',
    14: 'Solicitacao de cartao de remessas',
    15: 'Tempo de entrega do cartao',
    16: 'Tempo de remessa',
    17: 'Termos e condicoes do servico'
}

# Get the predicted class label
predicted_label = class_labels[predicted_class[0]]

print(f"Encoded and Padded Sentence: {padded_sentence}")
print(f"Predicted Class Index: {predicted_class[0]}")
print(f"Predicted Label: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Encoded and Padded Sentence: [[1264 1166 1468   92 1343 1074    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
Predicted Class Index: 0
Predicted Label: Acesso a conta


## Considerações finais

Para intuito de aprendizagem, seria interessante implementação de métricas como f1-score e recall. Como a proposta da instrução era um desafio de implementação com tempo limitado, o grupo não conseguiu finalizar.

Também fazia parte do desafio a implementação de visualização dos resultados obtidos utilizando Tensorboard, porém o grupo novamente foi alvo de tempo escasso.

Fontes utilizadas para implementação:
- [How to Use Word Embedding Layers for Deep Learning with Keras](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/)
- [Embedding Layer](https://keras.io/api/layers/core_layers/embedding/)
- ChatGPT